In [1]:
import requests
import re
from bs4 import BeautifulSoup
import html5lib
import multiprocessing.dummy as mp
URL = 'http://www.eprbullets.com/'

outdir = '../scrapes/'
outfile = outdir + 'eprbullets.txt'
homepage = requests.get(URL)

# need HTML5LIB because sometimes website has errors in HTML. html5lib is very forgiving.
homeSoup = BeautifulSoup(homepage.content,'html5lib')

In [2]:
pageList = homeSoup.find(id='mainNav').find_all('a', href=re.compile('\w+\.htm*'))
pageURLs = [URL + pageItem['href'] for pageItem in pageList]


In [3]:
def getBulletsInPage(pageURL, bulletDict):
    
    page = requests.get(pageURL)
    pageSoup = BeautifulSoup(page.content, 'html5lib')

    # Searches #bulletmain for all strings that match regex.
    bulletScrape = pageSoup.find(id="bulletmain").find_all(string=re.compile('-\s+.*'))
    bulletDict[pageURL]  = [bullet.strip() for bullet in bulletScrape]


In [4]:
# multiprocessing is used to speed up the scrape. 

# a dict/manager is used to collect all scraped bullets during multiprocessing.  
manager = mp.Manager()
resultsDict = manager.dict()

#running Pool() with no arguments will default to amount of cores you have... I think
p = mp.Pool()

p.starmap(getBulletsInPage, [(pageURL,resultsDict) for pageURL in pageURLs])
p.close()
p.join()

# Writing to file. Should default to clobbering.
with open(outfile,'w',encoding='utf-8') as fileID:
    for key in resultsDict:
        fileID.write(f'SECTION {key} \n')
        for line in resultsDict[key]:
            fileID.write(line + '\n')

In [5]:
# this block is for debugging
pageURL = 'http://www.eprbullets.com/ee.htm'
page = requests.get(pageURL)
pageSoup = BeautifulSoup(page.content, 'html5lib')
bulletScrape = pageSoup.find(id="bulletmain").find_all(string=re.compile('-\s.*$'))
bullets = [bullet.strip() for bullet in bulletScrape]